## **Sentiment Analysis**

## Importing Libraries

In [25]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
import kagglehub
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import warnings
warnings.filterwarnings('ignore')

## Importing Data 1

In [26]:
path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Maddox\.cache\kagglehub\datasets\abhi8923shriv\sentiment-analysis-dataset\versions\9


In [27]:
import os
files = os.listdir(path)
files

['test.csv',
 'testdata.manual.2009.06.14.csv',
 'train.csv',
 'training.1600000.processed.noemoticon.csv']

In [28]:
path1=path+'/'+files[0]
path2=path+'/'+files[1]
path3=path+'/'+files[2]
path4=path+'/'+files[3]
display(path1)

'C:\\Users\\Maddox\\.cache\\kagglehub\\datasets\\abhi8923shriv\\sentiment-analysis-dataset\\versions\\9/test.csv'

In [29]:
import pandas as pd
df1 = pd.read_csv(path1, encoding="iso-8859-1")

In [30]:
display(df1.sample(5))
display(df1.shape)
display(df1.columns)
display(df1.info())


,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
4204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1884,dd7a9fb210,spirit week! Tuesday have to be in make up by ...,negative,morning,46-60,Jamaica,2961167.0,10830.0,273.0
2280,5bdecb0d75,is still craving for a cheeseburger.,neutral,morning,46-60,Kuwait,4270571.0,17820.0,240.0
4150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(4815, 9)

Index(['textID', 'text', 'sentiment', 'Time of Tweet', 'Age of User',
       'Country', 'Population -2020', 'Land Area (Km²)', 'Density (P/Km²)'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            3534 non-null   object 
 1   text              3534 non-null   object 
 2   sentiment         3534 non-null   object 
 3   Time of Tweet     3534 non-null   object 
 4   Age of User       3534 non-null   object 
 5   Country           3534 non-null   object 
 6   Population -2020  3534 non-null   float64
 7   Land Area (Km²)   3534 non-null   float64
 8   Density (P/Km²)   3534 non-null   float64
dtypes: float64(3), object(6)
memory usage: 338.7+ KB


None

In [31]:
df2 = pd.read_csv(path2, encoding='utf-8')
display(df2.sample(2))
display(df2.shape)
display(df2.info())

,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,"@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right."
88,0,211,Mon May 25 17:21:08 UTC 2009,pelosi,zed01,Why the hell is Pelosi in freakin China? and o...
137,0,402,Tue Jun 02 03:01:10 UTC 2009,gm,mshbrown,It's unfortunate that after the Stimulus plan ...


(515, 6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 6 columns):
 #   Column                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                           --------------  ----- 
 0   4                                                                                                                515 non-null    int64 
 1   3                                                                                                                515 non-null    int64 
 2   Mon May 11 03:17:40 UTC 2009                                                                                     515 non-null    object
 3   kindle2                                                                                                          515 non-null    object
 4   tpryan                                       

None

In [32]:
df3 = pd.read_csv(path3, encoding="iso-8859-1")
display(df3.sample(5))
display(df3.shape)
display(df3.columns)
display(df3.info())

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
14129,baa4c5ad2b,Goin 2 eat some SMACKS 4 breakfast then gettin...,wish,positive,night,70-100,Zimbabwe,14862924,386850.0,38
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral,night,70-100,Antigua and Barbuda,97929,440.0,223
6044,ee267131b1,ok... twitter I almost pass out because of you...,****,negative,night,31-45,Iraq,40222493,434320.0,93
8636,89867c41e7,boston needs one too,boston needs one too,neutral,night,31-45,Iraq,40222493,434320.0,93
23761,e9c80bc8be,Just found out my kitty isn`t pregnant its a ...,good,positive,noon,21-30,Cyprus,1207359,9240.0,131


(27481, 10)

Index(['textID', 'text', 'selected_text', 'sentiment', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


None

In [33]:
df4 = pd.read_csv(path4, encoding="iso-8859-1")
display(df4.sample(5))
display(df4.shape)
display(df4.columns)
display(df4.info())

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
442293,0,2067236702,Sun Jun 07 11:59:41 PDT 2009,NO_QUERY,KCSuzy,I was watching diamond demos on MLB Network - ...
47484,0,1677646899,Sat May 02 03:12:56 PDT 2009,NO_QUERY,crunchybunny,wanting to go to my auntie's friend's despedid...
945695,4,1822395487,Sat May 16 19:28:50 PDT 2009,NO_QUERY,magza_,@amyyfrantz I assume that I am receiving one. ...
515985,0,2190964094,Tue Jun 16 04:11:25 PDT 2009,NO_QUERY,ClaireJulian,is still sore today and still bleeding!
981057,4,1834080192,Mon May 18 01:33:07 PDT 2009,NO_QUERY,Laura_McKay2009,@arengrimshaw Happy Monday at new desk!


(1048572, 6)

Index(['polarity of tweet ', 'id of the tweet', 'date of the tweet', 'query',
       'user', 'text of the tweet '],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048572 entries, 0 to 1048571
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   polarity of tweet   1048572 non-null  int64 
 1   id of the tweet     1048572 non-null  int64 
 2   date of the tweet   1048572 non-null  object
 3   query               1048572 non-null  object
 4   user                1048572 non-null  object
 5   text of the tweet   1048572 non-null  object
dtypes: int64(2), object(4)
memory usage: 48.0+ MB


None

## Importing Data 2 , 0-negative, 4=positive

In [34]:
path5 = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path5)

Path to dataset files: C:\Users\Maddox\.cache\kagglehub\datasets\kazanova\sentiment140\versions\2


In [35]:
fullpath=path5+'/'+os.listdir(path5)[0]
fullpath
df5 = pd.read_csv(fullpath, encoding="iso-8859-1")
display(df5.sample(5))
display(df5.shape)
display(df5.columns)
display(df5.info())

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
964856,4,1827558537,Sun May 17 10:58:06 PDT 2009,NO_QUERY,priscellie,"@Uilos ...Thank you, Craig, for ensuring that ..."
739843,0,2265831653,Sun Jun 21 07:23:47 PDT 2009,NO_QUERY,lizziemoogle,Im bored of cleaning now ... My back fucking k...
198281,0,1971185623,Sat May 30 06:27:19 PDT 2009,NO_QUERY,HannahFiore,aww watching the last episode with my mommy! o...
993014,4,1835327923,Mon May 18 05:44:00 PDT 2009,NO_QUERY,Dracona_,After going back over the timeline it seems I ...
274941,0,1990702757,Mon Jun 01 05:32:32 PDT 2009,NO_QUERY,michaelmknight,@dippyjax I wish It was a Bank Holiday


(1599999, 6)

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

None

In [36]:
display(df5.sample())

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
84307,0,1753565812,Sun May 10 02:16:49 PDT 2009,NO_QUERY,tattoogoddess,@rustyrockets i need a date to zombie prom in ...


## EDA on important dataset